In [129]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random

  Activating environment at `~/homework/optimal_control/planar_quadruped_landing/Project.toml`


In [130]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

reference_trajectory

## Reference Trajectory

## Problem Definition

In [131]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.02
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 0.2

n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
# xinit = [-0.4;0.6;-3.1415/9;        0.0;0.0; -1.0;0.2; 0.0;-0.5;0.0; 0.0;0.0; 0.0;-0.5]
# xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0; -lb;0.0;  0.0;0.0;0.0;  0.0;0.0; 0.0;0.0]
xinit = zeros(n)
xinit[1] = -lb/2                  # xb
xinit[2] = sqrt(l1^2+l2^2) + 0.05 # yb
xinit[3] = 10 * pi / 180          # theta
xinit[6] = -lb                    # x2
xinit[7] = 0.05                   # y2
xinit[9] = -3.0                   # yb_dot
xinit[12] = -3.0                  # y2_dot

xterm = zeros(n)
xterm[1] = -lb/2           # xb
xterm[2] = sqrt(l1^2+l2^2) # yb
xterm[6] = -lb             # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([1.0;1.0;1.0; 1.0;1.0; 1.0;1.0; 1.0;1.0;1.0; 1.0;1.0; 1.0;1.0])
R = Diagonal(fill(1e-3,4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [132]:
nlp.cinds

4-element Vector{UnitRange{Int64}}:
 1:14
 15:28
 29:448
 449:479

In [133]:
# Initial guess
xtransit = zeros(n)
xtransit[1] = -lb/2                  # xb
xtransit[2] = l1 # yb
xtransit[3] = 0          # theta
xtransit[6] = -lb     
Random.seed!(1)
Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Uguess = zeros(size(Uref))
Xguess = [x + 0.1*randn(length(x)) for x in Xref]
               # x2
timesteps_phase_1 = Int(t_trans / dt) 
for i = 1:timesteps_phase_1
    Xguess[i] = xinit + (xtransit - xinit)/timesteps_phase_1 * i
end
# display((xtransit - xinit)/timesteps_phase_1)
# @show timesteps_phase_1

In [134]:



Z0 = packZ(nlp, Xguess, Uguess);
# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...


This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   265366
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      554
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      479
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

([-0.24999999999991906, 0.4035533905933081, 0.17453292519943156, -5.5263578492761607e-14, 1.3241122696725608e-12, -0.5000000000000007, 0.0499999999999998, 9.894866202817922e-15, -2.9999999999999956, -6.796132080258427e-16  …  6.999295829537496e-17, -0.5000000000000152, -3.0408507286421578e-16, 3.734584910270216e-15, 1.817713522624629e-15, -7.301413877387352e-17, 1.2077980634864725e-15, -1.0030843991034412e-16, 2.2941804000712882e-15, -2.5830535640155665e-18], Ipopt.Optimizer)

In [135]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
  8.093525849517391e-14
  3.430589146091734e-14
 -1.3877787807814457e-15
 -5.5263578492761607e-14
  1.3241122696725608e-12
 -6.661338147750939e-16
 -2.0122792321330962e-16
  9.894866202817922e-15
  4.440892098500626e-15
 -6.796132080258427e-16
 -1.261704305478833e-15
  3.717026686445024e-13
 -3.04856747476586e-17
 -6.334997579206894e-17

In [136]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
 -7.216449660063518e-15
 -9.103828801926284e-15
  1.2811179925974138e-15
 -8.518644118587998e-15
  6.999295829537496e-17
 -1.5210055437364645e-14
 -3.0408507286421578e-16
  3.734584910270216e-15
  1.817713522624629e-15
 -7.301413877387352e-17
  1.2077980634864725e-15
 -1.0030843991034412e-16
  2.2941804000712882e-15
 -2.5830535640155665e-18

EXIT: Restoration failed!

EXIT: Converged to a point of local infeasibility. Problem may be infeasible.

In [137]:
display(Z_sol)
using DelimitedFiles
using CSV
writedlm( "data.csv",  Z_sol, ',')

554-element Vector{Float64}:
 -0.24999999999991906
  0.4035533905933081
  0.17453292519943156
 -5.5263578492761607e-14
  1.3241122696725608e-12
 -0.5000000000000007
  0.0499999999999998
  9.894866202817922e-15
 -2.9999999999999956
 -6.796132080258427e-16
  ⋮
 -0.5000000000000152
 -3.0408507286421578e-16
  3.734584910270216e-15
  1.817713522624629e-15
 -7.301413877387352e-17
  1.2077980634864725e-15
 -1.0030843991034412e-16
  2.2941804000712882e-15
 -2.5830535640155665e-18